In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# load excel file
file_path = os.path.join('raw_data','2007-2017-PIT-HUD.xlsx')
# read the excel file
df  = pd.read_excel(file_path, sheet_name=None, ignore_index=True)

# concatenate all worksheets
cdf = pd.concat(df.values())
# drop columns that have NaN
cdf = cdf.dropna(axis=1, how='any')

# arrange all columns
cdf = cdf[[ 'Year', 'CoC Number', 'CoC Name', 'Total Homeless', 'Sheltered Homeless', 'Unsheltered Homeless',
       'Homeless Individuals', 'Sheltered Homeless Individuals', 'Unsheltered Homeless Individuals', 
       'Homeless People in Families', 'Sheltered Homeless People in Families', 'Unsheltered Homeless People in Families',
       'Chronically Homeless Individuals', 'Sheltered Chronically Homeless Individuals', 
       'Unsheltered Chronically Homeless Individuals' ]]

# set the index to Year
cdf = cdf.set_index(['Year'])

# display consolidated data
cdf.head()

,CoC Number,CoC Name,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,Sheltered Homeless People in Families,Unsheltered Homeless People in Families,Chronically Homeless Individuals,Sheltered Chronically Homeless Individuals,Unsheltered Chronically Homeless Individuals
Year,,,,,,,,,,,,,,
2017,AK-500,Anchorage CoC,1128,973,155,848,693,155,280,280,0,114,70,44
2017,AK-501,Alaska Balance of State CoC,717,578,139,506,367,139,211,211,0,102,47,55
2017,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",1092,875,217,932,715,217,160,160,0,89,28,61
2017,AL-501,Mobile City & County/Baldwin County CoC,606,341,265,426,179,247,180,162,18,65,9,56
2017,AL-502,Florence/Northwest Alabama CoC,155,133,22,155,133,22,0,0,0,24,10,14


In [3]:
# Check record count before writing data into a csv file
cdf.count()

CoC Number                                      4358
CoC Name                                        4358
Total Homeless                                  4358
Sheltered Homeless                              4358
Unsheltered Homeless                            4358
Homeless Individuals                            4358
Sheltered Homeless Individuals                  4358
Unsheltered Homeless Individuals                4358
Homeless People in Families                     4358
Sheltered Homeless People in Families           4358
Unsheltered Homeless People in Families         4358
Chronically Homeless Individuals                4358
Sheltered Chronically Homeless Individuals      4358
Unsheltered Chronically Homeless Individuals    4358
dtype: int64

In [4]:
cdf.to_csv("HUD_Consol_Data.csv")